<a target="_blank" href="https://colab.research.google.com/github/sonder-art/automl_o24/blob/main/codigo/rag/rag_vector_db.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Enhanced RAG System with Vector Databases

This notebook implements a flexible RAG (Retrieval Augmented Generation) system with the following features:
- Configurable embedding models with GPU support
- Multiple document format support (txt, pdf, html, xml)
- Smart document processing with caching
- Flexible chunking strategies
- Efficient vector storage using LanceDB

## Setup and Installation

In [1]:
!pip install -q sentence-transformers lancedb pandas numpy beautifulsoup4 PyPDF2 tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.1/27.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.4/30.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00


In [24]:
import os
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import lancedb
import textwrap
from pathlib import Path
from typing import List, Optional, Dict, Union
import json
import hashlib
from bs4 import BeautifulSoup
import PyPDF2
from tqdm.auto import tqdm
import logging
from dataclasses import dataclass, asdict
import time
from datetime import datetime
import shutil
import tempfile
import os

In [5]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check if running in Colab and mount drive
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    from google.colab import drive
    # Mount drive explicitly before creating config
    drive.mount('/content/drive')
    logger.info("Google Drive mounted successfully")


Mounted at /content/drive


## Configuration

Enhanced configuration system with easy parameter modification

In [6]:
@dataclass
class RAGConfig:
    # Model Configuration
    embedding_model: str = "BAAI/bge-small-en-v1.5"  # Better default model
    use_gpu: bool = torch.cuda.is_available()
    device: str = "cuda" if use_gpu else "cpu"

    # Storage Configuration
    use_drive: bool = True if IN_COLAB else False  # Automatically use drive in Colab
    base_path: str = None  # Will be set in post_init

    # Chunking Configuration
    chunk_size: int = 500
    chunk_overlap: int = 50

    # Document Processing
    supported_formats: List[str] = None  # Will be set in post_init
    overwrite_existing: bool = False

    # Retrieval Configuration
    top_k: int = 3
    distance_metric: str = "cosine"

    def __post_init__(self):
        # Set base paths
        if self.use_drive and IN_COLAB:
            logger.info("Using Google Drive for storage...")
            self.base_path = Path('/content/drive/MyDrive/vector_db')
        else:
            self.base_path = Path('vector_store')

        self.docs_path = self.base_path / 'documents'
        self.db_path = self.base_path / 'db'

        # Create directories
        self.docs_path.mkdir(parents=True, exist_ok=True)
        self.db_path.mkdir(parents=True, exist_ok=True)

        # Set supported formats
        if self.supported_formats is None:
            self.supported_formats = ["txt", "pdf", "html", "xml"]

        logger.info(f"Using device: {self.device}")
        logger.info(f"Documents path: {self.docs_path}")
        logger.info(f"Database path: {self.db_path}")

    def save(self):
        """Save configuration to file"""
        config_data = asdict(self)
        config_data['base_path'] = str(self.base_path)
        config_data['docs_path'] = str(self.docs_path)
        config_data['db_path'] = str(self.db_path)

        config_path = self.base_path / 'config.json'
        with open(config_path, 'w') as f:
            json.dump(config_data, f, indent=2)
        logger.info(f"Configuration saved to {config_path}")

    @classmethod
    def load(cls, base_path: Optional[str] = None) -> 'RAGConfig':
        """Load configuration from file"""
        if base_path is None:
            if IN_COLAB:
                base_path = '/content/drive/MyDrive/vector_db'
            else:
                base_path = 'vector_store'

        config_path = Path(base_path) / 'config.json'
        if config_path.exists():
            with open(config_path) as f:
                config_data = json.load(f)
            logger.info(f"Configuration loaded from {config_path}")
            return cls(**config_data)
        logger.info("No existing configuration found, creating new one")
        return cls()

In [7]:
# Initialize configuration
config = RAGConfig()
config.save()

In [8]:

def verify_paths():
    logger.info("Verifying paths and contents:")
    for path in [config.base_path, config.docs_path, config.db_path]:
        logger.info(f"Path {path} exists: {path.exists()}")
        if path.exists():
            logger.info(f"Contents of {path}: {list(path.glob('*'))}")

verify_paths()

## Enhanced Document Processing

Improved document processor with multiple format support and smart caching

In [31]:
# Additional imports
import re
from bs4 import BeautifulSoup
import PyPDF2
import hashlib
import json
import logging
import os
import shutil
import tempfile
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Union
from tqdm.notebook import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer

class DocumentProcessor:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.model = SentenceTransformer(config.embedding_model, device=config.device)

    def _compute_file_hash(self, file_path: str) -> str:
        """Compute SHA-256 hash of a file."""
        sha256_hash = hashlib.sha256()
        with open(file_path, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()

    def _read_txt(self, file_path: str) -> str:
        """Read text from a txt file."""
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()

    def _read_pdf(self, file_path: str) -> str:
        """Read text from a PDF file."""
        text = ""
        with open(file_path, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            for page in pdf_reader.pages:
                text += page.extract_text() + "\n"
        return text

    def _read_html(self, file_path: str) -> str:
        """Read text from an HTML file."""
        with open(file_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'html.parser')
            return soup.get_text()

    def _read_xml(self, file_path: str) -> str:
        """Read text from an XML file."""
        with open(file_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'xml')
            return soup.get_text()

    def preprocess_text(self, text: str) -> str:
        """
        Preprocess text while maintaining proper spacing and formatting.
        """
        # Fix common OCR artifacts
        text = re.sub(r'ﬁ', 'fi', text)
        text = re.sub(r'ﬀ', 'ff', text)
        text = re.sub(r'ﬂ', 'fl', text)

        # Fix spacing issues
        text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)  # Add space between camelCase
        text = re.sub(r'(?<=[.!?])(?=[A-Z])', ' ', text)  # Add space after punctuation
        text = re.sub(r'[\n\r]+', ' ', text)  # Replace newlines with space

        # Normalize whitespace
        text = ' '.join(text.split())
        return text.strip()

    def process_text(self, text: str) -> List[str]:
        """Split text into overlapping chunks with improved text handling."""
        # First preprocess the text
        text = self.preprocess_text(text)

        # Split into sentences for better chunking
        sentences = re.split(r'(?<=[.!?])\s+', text)
        chunks = []
        current_chunk = []
        current_length = 0

        for sentence in sentences:
            sentence_length = len(sentence.split())

            if current_length + sentence_length > self.config.chunk_size and current_chunk:
                # Join the current chunk and add it to chunks
                chunks.append(' '.join(current_chunk))
                # Keep last sentence for overlap if overlap is enabled
                if self.config.chunk_overlap > 0 and current_chunk:
                    current_chunk = current_chunk[-1:]
                    current_length = len(current_chunk[0].split())
                else:
                    current_chunk = []
                    current_length = 0

            current_chunk.append(sentence)
            current_length += sentence_length

        # Add the last chunk if it's not empty
        if current_chunk:
            chunks.append(' '.join(current_chunk))

        return chunks

    def process_file(self, file_path: str) -> Dict:
        """Process a single file with caching support."""
        file_ext = file_path.split('.')[-1].lower()
        if file_ext not in self.config.supported_formats:
            raise ValueError(f"Unsupported file format: {file_ext}")

        # Check cache
        file_hash = self._compute_file_hash(file_path)
        cache_path = self.config.docs_path / f"{file_hash}.json"

        if cache_path.exists() and not self.config.overwrite_existing:
            logger.info(f"Loading cached processing for {file_path}")
            with open(cache_path) as f:
                return json.load(f)

        # Read content based on file type
        readers = {
            'txt': self._read_txt,
            'pdf': self._read_pdf,
            'html': self._read_html,
            'xml': self._read_xml
        }

        text = readers[file_ext](file_path)
        chunks = self.process_text(text)

        # Use GPU for batch processing if available
        embeddings = self.model.encode(chunks, show_progress_bar=True)

        result = {
            'chunks': chunks,
            'embeddings': embeddings.tolist(),
            'source': str(file_path),
            'file_hash': file_hash
        }

        # Cache result
        with open(cache_path, 'w') as f:
            json.dump(result, f)

        return result

    def process_directory(self, dir_path: Optional[str] = None, table_name: str = 'documents') -> None:
        """Process all files in a directory and handle Google Drive limitations."""
        start_time = time.time()
        dir_path = Path(dir_path) if dir_path else self.config.docs_path

        # Create a temporary directory for processing
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_db_path = Path(temp_dir) / 'temp_db'
            logger.info(f"Created temporary database at: {temp_db_path}")

            # Connect to temporary database
            db = lancedb.connect(temp_db_path)

            logger.info(f"Starting document processing at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            logger.info(f"Scanning directory: {dir_path}")
            logger.info(f"Supported formats: {', '.join(self.config.supported_formats)}")

            # Rest of the process_directory method remains unchanged...
            # [Previous process_directory implementation continues here]

            files_to_process = []
            for ext in self.config.supported_formats:
                files = list(dir_path.glob(f"**/*.{ext}"))
                if files:
                    logger.info(f"Found {len(files)} .{ext} files")
                files_to_process.extend(files)

            if not files_to_process:
                logger.warning("No supported files found in directory")
                return

            logger.info(f"Total files to process: {len(files_to_process)}")

            all_data = []
            successful_files = 0
            failed_files = 0
            total_chunks = 0

            for file_path in tqdm(files_to_process, desc="Processing files", unit="file"):
                try:
                    logger.info(f"\nProcessing {file_path}...")
                    result = self.process_file(str(file_path))

                    num_chunks = len(result['chunks'])
                    logger.info(f"Generated {num_chunks} chunks from {file_path.name}")

                    for chunk, embedding in zip(result['chunks'], result['embeddings']):
                        all_data.append({
                            'text': chunk,
                            'vector': embedding,
                            'source': result['source'],
                            'file_hash': result['file_hash']
                        })

                    successful_files += 1
                    total_chunks += num_chunks

                except Exception as e:
                    failed_files += 1
                    logger.error(f"Error processing {file_path}: {str(e)}")

            if all_data:
                logger.info("\nPreparing to update database...")
                df = pd.DataFrame(all_data)

                logger.info(f"Creating new table with {len(df)} chunks...")
                with tqdm(total=1, desc="Creating table") as pbar:
                    db.create_table(table_name, df)
                    pbar.update(1)

                logger.info("Copying database to Google Drive...")
                try:
                    os.makedirs(self.config.db_path, exist_ok=True)

                    for item in tqdm(os.listdir(temp_db_path), desc="Copying to Drive"):
                        src_path = os.path.join(temp_db_path, item)
                        dst_path = os.path.join(self.config.db_path, item)
                        if os.path.isdir(src_path):
                            shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
                        else:
                            shutil.copy2(src_path, dst_path)

                    logger.info("Database successfully copied to Google Drive")
                except Exception as e:
                    logger.error(f"Error copying to Google Drive: {str(e)}")
                    logger.error(f"Temporary database remains at: {temp_db_path}")
                    raise

            end_time = time.time()
            processing_time = end_time - start_time

            logger.info("\n" + "="*50)
            logger.info("Processing Summary:")
            logger.info(f"Total files processed: {len(files_to_process)}")
            logger.info(f"Successfully processed: {successful_files}")
            logger.info(f"Failed to process: {failed_files}")
            logger.info(f"Total chunks generated: {total_chunks}")
            logger.info(f"Average chunks per file: {total_chunks/successful_files if successful_files else 0:.1f}")
            logger.info(f"Total processing time: {processing_time:.1f} seconds")
            logger.info(f"Average time per file: {processing_time/len(files_to_process):.1f} seconds")
            logger.info("="*50)

## Enhanced Vector Database Operations

Improved vector database class with better search capabilities

In [27]:
from typing import List, Dict, Optional
import pandas as pd
import lancedb
from sentence_transformers import SentenceTransformer
import logging
import textwrap
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class VectorDB:
    def __init__(self, config: RAGConfig):
        self.config = config
        self.db = lancedb.connect(config.db_path)
        self.model = SentenceTransformer(config.embedding_model, device=config.device)

    def list_tables(self) -> List[str]:
        """List all tables in the database."""
        return self.db.table_names()

    def get_table_info(self, table_name: str) -> Dict:
        """Get detailed information about a table."""
        table = self.db.open_table(table_name)
        df = table.to_arrow().to_pandas()

        if df.empty:
            return {
                'total_chunks': 0,
                'unique_documents': 0,
                'sources': [],
                'chunks_per_source': {},
                'avg_chunk_length': 0
            }

        sources = df['source'].unique()
        source_counts = df['source'].value_counts().to_dict()

        return {
            'total_chunks': len(df),
            'unique_documents': len(sources),
            'sources': sources.tolist(),
            'chunks_per_source': source_counts,
            'avg_chunk_length': df['text'].str.len().mean()
        }

    def semantic_search(self,
                       query: str,
                       table_name: str = 'documents',
                       k: Optional[int] = None,
                       threshold: Optional[float] = None,
                       source_filter: Optional[List[str]] = None) -> pd.DataFrame:
        """Enhanced semantic search with filtering options."""
        k = k or self.config.top_k
        query_embedding = self.model.encode([query])[0]
        table = self.db.open_table(table_name)

        # Build search query
        search_query = table.search(query_embedding)

        # Apply source filter if provided
        if source_filter:
            # Properly format the list for the SQL-like query
            formatted_sources = "(" + ", ".join([f"'{s}'" for s in source_filter]) + ")"
            search_query = search_query.where(f"source IN {formatted_sources}")

        # Execute the search and convert to Pandas DataFrame
        results_arrow = search_query.limit(k).to_arrow()

        # Convert to Pandas DataFrame
        results = results_arrow.to_pandas()

        # Debug: Print available columns
        print("Available columns in search results:", results.columns.tolist())

        if len(results) == 0:
            logger.warning("No results found matching the criteria")
            return pd.DataFrame()

        # Compute cosine similarity manually
        # Ensure 'vector' column exists and contains embeddings
        if 'vector' not in results.columns:
            logger.error("No 'vector' column found in the search results.")
            return pd.DataFrame()

        db_embeddings = np.array(results['vector'].tolist())
        query_embedding_np = np.array(query_embedding).reshape(1, -1)
        similarities = cosine_similarity(db_embeddings, query_embedding_np).flatten()

        # Add similarity scores to the DataFrame
        results['similarity'] = similarities

        # Apply similarity threshold if provided
        if threshold is not None:
            results = results[results['similarity'] >= threshold]
            if results.empty:
                logger.warning("No results meet the similarity threshold.")
                return pd.DataFrame()

        # Sort results by similarity in descending order
        results = results.sort_values('similarity', ascending=False)

        # Select and rename relevant columns
        return results[['text', 'source', 'similarity']]

    def batch_search(self,
                    queries: List[str],
                    table_name: str = 'documents',
                    k: Optional[int] = None) -> List[pd.DataFrame]:
        """Perform batch semantic search for multiple queries."""
        k = k or self.config.top_k

        # Batch encode queries
        query_embeddings = self.model.encode(queries, show_progress_bar=True)

        results = []
        table = self.db.open_table(table_name)

        for query, embedding in zip(queries, query_embeddings):
            search_query = table.search(embedding).limit(k)
            results_arrow = search_query.to_arrow()
            df = results_arrow.to_pandas()

            if df.empty:
                logger.warning(f"No results found for query: {query}")
                results.append(pd.DataFrame())
                continue

            # Compute cosine similarity manually
            if 'vector' not in df.columns:
                logger.error(f"No 'vector' column found in search results for query: {query}")
                results.append(pd.DataFrame())
                continue

            db_embeddings = np.array(df['vector'].tolist())
            embedding_np = np.array(embedding).reshape(1, -1)
            similarities = cosine_similarity(db_embeddings, embedding_np).flatten()

            # Add similarity scores to the DataFrame
            df['similarity'] = similarities

            # Sort by similarity in descending order
            df = df.sort_values('similarity', ascending=False)

            # Add the query to the results
            df['query'] = query
            results.append(df[['query', 'text', 'source', 'similarity']])

        return results

    def delete_table(self, table_name: str) -> None:
        """Delete a table from the database."""
        if table_name in self.list_tables():
            self.db.drop_table(table_name)
            logger.info(f"Table '{table_name}' deleted")
        else:
            logger.warning(f"Table '{table_name}' not found")


## Example Usage

This code deletes the current table you have chose.

In [44]:
def cleanup_rag_system(config: RAGConfig, include_documents: bool = False) -> None:
    """
    Clean up the RAG system by removing database, embeddings, and optionally documents.

    Args:
        config: RAGConfig instance containing path information
        include_documents: If True, also removes the original documents. Default is False.
    """
    try:
        # 1. Remove the database
        if config.db_path.exists():
            logger.info(f"Removing database at {config.db_path}")
            shutil.rmtree(config.db_path)
            logger.info("Database removed successfully")
        else:
            logger.info("No database found to remove")

        # 2. Remove cached embeddings (json files)
        cache_files = list(config.docs_path.glob("*.json"))
        if cache_files:
            logger.info(f"Removing {len(cache_files)} cached embedding files")
            for cache_file in cache_files:
                cache_file.unlink()
            logger.info("Cache files removed successfully")
        else:
            logger.info("No cache files found to remove")

        # 3. Optionally remove the documents
        if include_documents:
            if config.docs_path.exists():
                # Only remove the contents, not the directory itself
                for item in config.docs_path.iterdir():
                    if item.is_file() and item.suffix.lower()[1:] in config.supported_formats:
                        item.unlink()
                logger.info("Documents removed successfully")
            else:
                logger.info("No documents found to remove")

        logger.info("Cleanup completed successfully")

        # 4. Recreate necessary directories
        config.docs_path.mkdir(parents=True, exist_ok=True)
        config.db_path.mkdir(parents=True, exist_ok=True)
        logger.info("Directory structure recreated")

    except Exception as e:
        logger.error(f"Error during cleanup: {str(e)}")
        raise

In [45]:

# To clean up everything except the original documents:
# # cleanup_rag_system(config)

# # # # To clean up everything including the original documents:
# # # # cleanup_rag_system(config, include_documents=True)

In [46]:
# Initialize system with custom configuration if needed
config = RAGConfig(
    embedding_model="BAAI/bge-small-en-v1.5",  # Change model if desired
    chunk_size=300,  # Adjust chunk size
    chunk_overlap=30,  # Adjust overlap
    top_k=5  # Adjust number of results
)

# Initialize processor and process documents
processor = DocumentProcessor(config)
processor.process_directory()

# Initialize vector database
db = VectorDB(config)



Processing files:   0%|          | 0/7 [00:00<?, ?file/s]

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Creating table:   0%|          | 0/1 [00:00<?, ?it/s]

Copying to Drive:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
# Get database information
print("\nAvailable tables:")
for table_name in db.list_tables():
    info = db.get_table_info(table_name)
    print(f"\nTable: {table_name}")
    print(f"Total chunks: {info['total_chunks']}")
    print(f"Unique documents: {info['unique_documents']}")
    print(f"Average chunk length: {info['avg_chunk_length']:.0f} characters")



Available tables:

Table: documents
Total chunks: 5881
Unique documents: 6
Average chunk length: 2441 characters


In [48]:
# Example semantic search
query = '''should i use all the data avaialble with the hyperparameters from cross validation to train the complete dataset after selecting them?'''
results = db.semantic_search(
    query=query,
    k=3,  # Number of results
    threshold=0.2,  # Optional similarity threshold
    source_filter=None  # Optional source filtering
)

print(f"\nQuery: {query}\n")
for _, row in results.iterrows():
    print(f"Score: {row['similarity']:.4f}")
    print(f"Source: {row['source']}")
    print(f"Text: {textwrap.fill(row['text'], width=80)}\n")


Available columns in search results: ['text', 'vector', 'source', 'file_hash', '_distance']

Query: should i use all the data avaialble with the hyperparameters from cross validation to train the complete dataset after selecting them?

Score: 0.7485
Source: /content/drive/MyDrive/vector_db/documents/ESLII.pdf
Text: If we are in a data-rich situation, the best approach for both problems is to
randomly divide the dataset into three parts: a training set, a validation set,
and a test set. The training set is used to fit the models; the validation set
is used to estimate prediction error for model selection; the test set is used
for assessment of the generalization error of the final chosen model. Ideally,
the test set should be kept in a “vault,” and be brought out only at the end of
the data analysis. Suppose instead that we use the test-set repeatedly, choosing
the model with smallest test-set error. Then the test set error of the final
chosen model will underestimate the true test erro

In [ ]:
# Example batch search
queries = ["query 1", "query 2", "query 3"]
batch_results = db.batch_search(queries)

for df in batch_results:
    print(f"\nResults for query: {df['query'].iloc[0]}")
    for _, row in df.iterrows():
        print(f"Score: {row['similarity']:.4f}")
        print(f"Text: {textwrap.fill(row['text'], width=80)}\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Results for query: query 1
Score: 0.1608
Text: the use of grocery store scanner data, allowance for nonparametric, general
heterogeneity in the cross-section, including zeros in regressions, and in the
comparison of cross-section and panelresults. 2 Demand and Weighted Average
Surplus We consider a demand model where the form of heterogeneity is
unrestricted. To describe themodel let denote the quantity of a vector of
goods, the quantity of a numeraire good, the price vector for relative to ,a
n dthe individual income level relative to the numeraire price. The unobserved
heterogeneity will be represented by a vector of unobserved disturbances of
unknown dimension. We think of each value of as corresponding to a consumer but
do allow to be continuously distributed. For each consumer the demand function
( )will be obtained by maximizing a utility function( )that is
monotonic increasing in andsubject to the budget constraint, with ( ) =
arg max ≥0≥0(